# Setting up an OpenMM iMD simulation with ASE and NanoVer

This notebook demonstrates how to set up an OpenMM simulation for use with NanoVer from scratch.
We take AMBER files for neuraminidase with oseltamivir (AKA tamiflu) bound, create an OpenMM system and 
set it up with NanoVer, using ASE as the integrator for the interactive molecular dynamics (iMD) simulation. 

## Set up the OpenMM Simulation

We start by creating an OpenMM simulation for the neuraminidase-oseltamivir system using the relevant pre-prepared AMBER files. OpenMM also supports Gromacs and CHARMM files, and can be customized for many other uses. 

In [1]:
import openmm as mm
import openmm.unit as unit 
import openmm.app as app

In [2]:
prmtop = app.AmberPrmtopFile("openmm_files/3TI6_ose_wt.top")
amber_coords = app.AmberInpcrdFile("openmm_files/3TI6_ose_wt.rst")

Because we are going to use ASE to integrate the equations of motion, we keep the simulation simple by using implicit solvent and no constraints.

In [3]:
system = prmtop.createSystem(nonbondedMethod=app.CutoffPeriodic, 
                             nonbondedCutoff=2*unit.nanometer, 
                             implicitSolvent=app.OBC2,
                             constraints=None)

/opt/homebrew/Caskroom/miniforge/base/envs/nanover-dev/lib/python3.12/site-packages/openmm/app/internal/amber_file_parser.py:1168: UserWarning: Non-optimal GB parameters detected for GB model OBC2
  warnings.warn(


To construct an input file for an OpenMM simulation in NanoVer, we need to define an integrator. We will also use this integrator to equilibrate the simulation before saving it to an input file. However, *these parameters are not currently passed to ASE* (as the set of integrators offered by OpenMM and ASE differ), so we will need to define the ASE integrator explicitly later on.

In [4]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.001*unit.picoseconds)

Now we create an OpenMM simulation from the topology, system and integrator that we have defined, and define the positions using the AMBER input coordinate file.

In [5]:
simulation = app.Simulation(prmtop.topology, system, integrator)

In [6]:
simulation.context.setPositions(amber_coords.positions)
if amber_coords.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*amber_coords.boxVectors)

Let's minimize the energy to create a stable conformation that can be used to run dynamics:

In [7]:
simulation.minimizeEnergy()

Now let's run a few steps to check it's stable, printing the potential energy and temperature every 500 steps.

In [8]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)

In [9]:
import sys

In [10]:
simulation.reporters.append(app.StateDataReporter(sys.stdout, 500, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(5000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
500,-35892.80851004743,208.75830794933293
1000,-33098.39441702985,243.0947096638716
1500,-31567.636558753482,265.3980710461487
2000,-30357.08402274274,277.1831587655949
2500,-29505.31430838727,283.0251879352551
3000,-29421.981117469302,292.1731576697053
3500,-29400.87766287946,295.71076744311154
4000,-28972.899574500552,292.206369629022
4500,-28670.388008338443,297.33164775276924
5000,-28895.867256385318,299.738964317865


Looks good! Now, let's set it up for use with NanoVer.

The following cell outputs the system as an OpenMM XML file + a PDB file with the topology. This let's you save what you've done here to an input file, so you can load the simulation immediately for use in NanoVer without having to reconstruct the simulation.

In [11]:
from nanover.openmm.serializer import serialize_simulation

with open('neuraminidase_nanover.xml','w') as f:
    f.write(serialize_simulation(simulation))

In the following section, we'll set up the simulation by hand, essentially replicating what `nanover-omm-ase` does.

## Setting up an OpenMM simulation with ASE and NanoVer

Now that we've got an OpenMM simulation, let's pair it with ASE so we can do interactive molecular dynamics with NanoVer. 

This setup uses both ASE and OpenMM to perform the iMD simulation:
* OpenMM calculates the forces acting on the system (i.e. the "calculator" for ASE defined in NanoVer)
* ASE performs the integration of the equations of motion

**Note**: The `nanover-openmm` package lets you use the inbuilt integrators in OpenMM to perform iMD simulations. These types of are defined using the `OpenMMSimulation` class: for more information on this approach, see the tutorials in the [OpenMM directory](../openmm).

### Creating the ASE System

We use the `OpenMMCalculator` class to take our simulation and produce an ASE Atoms object. We then assign the `OpenMMCalculator` as the calculator for the simulation—this tells ASE that we're using OpenMM to calculate the forces that ASE will use to integrate the equations of motion.

In [12]:
from nanover.ase.omm_calculator import OpenMMCalculator

In [13]:
# Define the calculator using the OpenMM simulation defined above
calculator = OpenMMCalculator(simulation)

In [14]:
# Define the atoms object and set it's calculator as the OpenMMCalculator
atoms = calculator.generate_atoms()
atoms.calc = calculator
len(atoms)

5880

Now we've got an ASE Atoms object, and a calculator, we can set up NanoVer with ASE as usual. 
The key difference between this setup and the [basic ASE tutorial](./ase_basic_example.ipynb) is that we swap out the default way of sending frames with a specially made one, `calculator.make_frame_adaptor()`, that knows about OpenMM topology.

In [15]:
from ase.md import Langevin
import ase.units as ase_units
dynamics = Langevin(atoms, timestep=1.0 * ase_units.fs, temperature_K=300, friction=1.0e-03)

In [16]:
from nanover.omni import OmniRunner
from nanover.omni.ase import ASESimulation

In [17]:
ase_sim = ASESimulation.from_ase_dynamics(
    dynamics, 
    ase_atoms_to_frame_data=calculator.make_frame_adaptor(),
)
imd_runner = OmniRunner.with_basic_server(ase_sim)

As always, let's run a few steps to make sure everything looks good

In [18]:
ase_sim.dynamics.run(100)
ase_sim.atoms.get_potential_energy()

-410.4492109598051

All good! Let's leave the simulation running in the background

In [19]:
imd_runner.next()

Connect to it from VR and you'll see something like this:

![NanoVer neuraminidase](./images/neuraminidase_ball_and_stick.png)

## Let's make it pretty!

Ball and stick is so 2001, let's make it look cool. We'll also make it so if you interact with the oseltamivir, it'll be interacted with as a group, which is more stable

First, we connect a client so we can modify the shared state, which contains the information defining how NanoVer should render the system for VR clients connecting to the server.

In [20]:
from nanover.app import NanoverImdClient
client = NanoverImdClient.connect_to_single_server(port=imd_runner.app_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame();

We define a couple of handy methods for playing with selections and colour gradients

In [21]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.colormaps[name]
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

In [22]:
from nanover.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Hide the 'root' selection, it's getting in the way of our creativity 

In [23]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

Let's select the protein

In [24]:
protein = client.create_selection("Protein", [])

In [25]:
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

We'll colour it and render with a spline, or ribbon, renderer.
Some things you can try: 
* Change the render: `spline`, `geometric spline`. Or comment out the `sequence` line and try `liquorice`,`noodles`, `cycles`, `ball and stick`.
* Change the color: set it to be one color, or try some different matplotlib [color maps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html), e.g. `rainbow` or `magma`.
* Change the scale.

In [26]:
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('rainbow')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }
    protein.interaction_method = 'single'

Let's reintroduce the ligand, oseltamivir, and make it so we interact with it as a group 

In [27]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [28]:
with ligand.modify():
    ligand.renderer = {
            'color': 'cpk',
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligand.velocity_reset = True
    ligand.interaction_method = 'group'

If you've done all that, you'll have something that looks like this:

![Neuraminidase Geometric](./images/neuraminidase_geometric_spline.png)

# Tidying Up After Yourself

Once you are finished with the iMD simulation, it's good practice to close both the server and the clients that connect to it. You can close the python client and server by calling `.close()` on them:

In [29]:
client.close()
imd_runner.close()

# Next Steps

In this tutorial, we demonstrated how to set up an OpenMM simulation, how to use that file to set up an iMD simulation using ASE (with an OpenMM calculator) to integrate the equations of motion, and how to alter the representation of the system for VR clients. From here, we recommend checking out the following tutorials:
* Set up an ASE/OpenMM simulation of [graphene](./ase_openmm_graphene.ipynb) with restraints and add UI and custom commands in the notebook.
* Learn how to perform iMD simulations using OpenMM with NanoVer by checking out the [NanoVer OpenMM tutorials](../openmm).